# Database Engineering
Now that the data is cleaned, it is a good idea to store it in a relational database. This will allow for a faster access and easier, more flexible query.

We will accomplish this step using [SQLAlchemy](https://www.sqlalchemy.org/), a database toolkit for Python.

SQLAlchemy is an ORM (object-relational mapper) allows you do easily interact with relational databases by creating a mapping from a Python class object to a relational database table.

It works by creating Python classes, one for each table in the database, whose attributes map to columns in the table. Once the schema is outlined by writing these classes, it is a simple matter to convert Python objects to records in the relational database, and vice-versa. 

In SQLAlchemy's words:

>"The SQLAlchemy Object Relational Mapper presents a method of associating user-defined Python classes with database tables, and instances of those classes (objects) with rows in their corresponding tables. It includes a system that transparently synchronizes all changes in state between objects and their related rows, called a unit of work, as well as a system for expressing database queries in terms of the user defined classes and their defined relationships between each other."

This will make more sense as we go along, but for a full run-down, check the [SQLAlchemy Documentation](https://docs.sqlalchemy.org/en/latest/), especially the [Object Relational Tutorial](https://docs.sqlalchemy.org/en/latest/orm/tutorial.html).

### Dependencies

In [1]:
from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Float

### Create the base class
User-defined classes in SQLAlchemy inherit attributes of a base class, which must first be instantiated:

In [2]:
Base = declarative_base()

### Set up ORM templates
Here we will declare our mapping from classes to the database tables. Within the class, we define details about the table to which we'll be mapping, primarily the table name, and names and datatypes of the columns.

In [3]:
class Stations(Base):
    __tablename__ = 'hawaii_stations'
    id = Column(Integer, primary_key=True)
    station = Column(String)
    name = Column(String)
    latitude = Column(Float)
    longitude = Column(Float)
    elevation = Column(Float)
    
class Measurement(Base):
    __tablename__ = 'hawaii_measurements'
    id = Column(Integer, primary_key=True)
    station = Column(String)
    date = Column(String)
    prcp = Column(Float)
    tobs = Column(Integer)

### Create and Connect to hawaii.sqlite database
`create_engine()` returns an instance of `Engine`, which is used by SQLAlchemy behind the scenes to connect to and interact with the database. Here we supply a sqlite database url, which SQLAlchemy will create if it doesn't yet exist.

In [4]:
engine = create_engine('sqlite:///hawaii.sqlite')
conn = engine.connect()

### Initiate mapping with create_all
The next step is to use the above `Engine` instance to connect to the database, and tell SQLAlchemy to emit the required SQL to construct the schema we defined with the above classes:

In [5]:
Base.metadata.create_all(engine)

Now the SQLite database is set up, and ready for the insertion of data.

### Creating a Session
Now that the database schema has been constructed, it is time to start talking to the database.

In order to manage transactions with the database, it is best to create a "session". During this session, SQLAlchemy tracks changes to objects, and provides all of the plumbing to persist changes to the database.

The `session` object acts as a handle to the database.

In [6]:
from sqlalchemy.orm import Session
session = Session(bind=engine)

## Converting `.csv` Rows to Class Objects
Remember that SQLAlchemy works by making a mapping from Python class objects to rows of a table. In order to reflect the objects into the database, they first must be constructed.

To easily accomplish this, we will:
1. Read in `.csv`s and convert them to PANDAS dataframes
2. Use PANDAS function `.to_dict()` with the `orient='records'` kwarg, to convert each dataframe into a list of dictionaries (each dictionary representing one row)
3. Loop over the dictionaries, each time instantiating a new class object of the appropriate class.

### Read in `.csv`s and Create Dataframes

In [7]:
import pandas as pd
csv1 = pd.read_csv('Output/stations_clean.csv')
csv2 = pd.read_csv('Output/measurements_clean.csv')
stations = pd.DataFrame(csv1)
measurements = pd.DataFrame(csv2)

### Convert Dataframes to Dictionaries

In [8]:
# http://pandas-docs.github.io/pandas-docs-travis/io.html#orient-options
station_dict = csv1.to_dict(orient='records')
measurement_dict = csv2.to_dict(orient='records')

In [9]:
station_dict[0]

{'elevation': 3.0,
 'latitude': 21.2716,
 'longitude': -157.8168,
 'name': 'WAIKIKI 717.2, HI US',
 'station': 'USC00519397'}

In [10]:
measurement_dict[0]

{'date': '2010-01-01', 'prcp': 0.08, 'station': 'USC00519397', 'tobs': 65}

## Populating the Tables
Now we loop over the list of dictionaries, instantiating a new member of the appropriate class each time.

`session.add()` stages the class object for reflection in the database, and `session.commit()` flushes all staged objects to the database.

`**` preceding the dictionary instructs Python to unpack the dictionary, and supply the values as keyword arguments for class object instantiation.

In [11]:
for station in station_dict:
    prepped_station = Stations(**station)
    session.add(prepped_station)

In [12]:
for measurement in measurement_dict:
    prepped_measurement = Measurement(**measurement)
    session.add(prepped_measurement)

In [13]:
session.commit()

## Querying the Database
To test, we will query the database and print out some contents:

In [14]:
station_list = session.query(Stations)
for station in station_list:
    print(station.name)

WAIKIKI 717.2, HI US
KANEOHE 838.1, HI US
KUALOA RANCH HEADQUARTERS 886.9, HI US
PEARL CITY, HI US
UPPER WAHIAWA 874.3, HI US
WAIMANALO EXPERIMENTAL FARM, HI US
WAIHEE 837.5, HI US
HONOLULU OBSERVATORY 702.2, HI US
MANOA LYON ARBO 785.2, HI US


In [19]:
session.query(Measurement).first().tobs

65

In [20]:
session.close()